# Project Library

In [1]:
# | default_exp asana.project

In [2]:
# | exporti

from __future__ import annotations

from dataclasses import dataclass, field
from typing import List
import datetime as dt

from nbdev.showdoc import patch_to

import domolibrary_extensions.utils.utils as ut
import domolibrary_extensions.client as gd
import domolibrary_extensions.asana.auth as aa
import domolibrary_extensions.asana.user as au

In [3]:
# |hide
from dotenv import load_dotenv
import os
import json

assert load_dotenv("../.env")

In [4]:
# | export
@dataclass
class AsanaProject:
    id: str
    name: str
    workspace_id: str
    permalink_url: str
    auth: aa.AsanaAuth = field(repr=False)

    is_archived: bool
    is_completed: bool

    created_date: dt.datetime
    modified_date: dt.datetime

    owner: au.AsanaUser
    members: List[au.AsanaUser]

    due_date: dt.datetime = None
    completed_date: dt.datetime = None

    tasks = None  # AsanaTask

    @classmethod
    def _from_json(cls, obj, auth: aa.AsanaAuth):
        owner = (
            au.AsanaUser._from_json(obj.get("owner"), auth=auth)
            if obj.get("owner")
            else None
        )

        members = (
            [
                au.AsanaUser._from_json(member, auth=auth)
                for member in obj.get("members", [])
            ]
            if obj.get("members")
            else None
        )

        created_date = ut.convert_str_to_date(obj.get("created_at"))
        modified_date = ut.convert_str_to_date(obj.get("modified_at"))
        due_date = ut.convert_str_to_date(obj.get("due_on"))
        completed_date = ut.convert_str_to_date(obj.get("completed_at"))

        return cls(
            auth=auth,
            id=obj.get("gid"),
            name=obj.get("name"),
            workspace_id=obj.get("workspace", {}).get("gid"),
            permalink_url=obj.get("permalink_url"),
            is_archived=obj.get("archived"),
            is_completed=obj.get("completed"),
            created_date=created_date,
            modified_date=modified_date,
            owner=owner,
            members=members,
            due_date=due_date,
            completed_date=completed_date,
        )

In [5]:
# |exporti
@patch_to(AsanaProject, cls_method=True)
async def get_projects(
    cls: AsanaProject,
    auth: aa.AsanaAuth,
    debug_api: bool = False,
    return_raw: bool = False,
):
    url = f"{auth.base_url}/projects"

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return [
        AsanaProject._from_json(proj_obj, auth=auth)
        for proj_obj in res.response["data"]
    ]

#### sample implementation of get_projects

In [6]:
# destructure auth fields from env

asana_pat = json.loads(os.environ["ASANA_PUBLIC"])["ASANA_PAT"]

workspace_id = "1206043185525448"
user_id = "1206043147237379"
asana_auth = aa.AsanaAuth(token=asana_pat, workspace_id=workspace_id)

asana_projects = await AsanaProject.get_projects(
    auth=asana_auth, debug_api=True, return_raw=False
)
asana_projects

[AsanaProject(id='1206043185525448', name='asana_sync', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None)]

In [7]:
# | exporti
@patch_to(AsanaProject, cls_method=True)
async def get_by_id(
    cls: AsanaProject,
    auth: aa.AsanaAuth,
    project_id,
    debug_api: bool = False,
    return_raw: bool = False,
):
    url = f"{auth.base_url}/projects/{project_id}"

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return cls._from_json(res.response["data"], auth=auth)

#### sample implementation of get_project_by_id

In [8]:
project_id = asana_projects[0].id

asana_project = await AsanaProject.get_by_id(auth=asana_auth, project_id=project_id)
asana_project

AsanaProject(id='1206043185525448', name='asana_sync', workspace_id='1206043147237389', permalink_url='https://app.asana.com/0/1206043185525448/1206043185525448', is_archived=False, is_completed=False, created_date=datetime.datetime(2023, 11, 27, 22, 34, 13, 86000, tzinfo=tzutc()), modified_date=datetime.datetime(2023, 11, 27, 22, 49, 29, 374000, tzinfo=tzutc()), owner=AsanaUser(id='1206043147237379', name='Jae Wilson', email=None), members=[AsanaUser(id='1206043147237379', name='Jae Wilson', email=None)], due_date=None, completed_date=None)

In [9]:
# | hide

import nbdev

nbdev.nbdev_export()